#### Dependencies

In [ ]:
!pip install tensorflow tensorflow-cpu opencv-python matplotlib

In [ ]:
!pip list

#### Importing Tensorflow and lessening GPU consumption

In [ ]:
import tensorflow as tf
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

#### Importing Opencv and removing dogy images

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'data' 

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

#### Loading Training data

In [ ]:
import numpy as np

In [ ]:
data = tf.keras.preprocessing.image_dataset_from_directory(
    directory='data',
    validation_split=0.3,
    subset='training',
    seed=123,
    image_size=(256, 256),
    batch_size=32,
)

#### Scaling data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

#### Splitting data

In [ ]:
train_data = data.skip(int(0.3 * len(data)))


In [ ]:
train_data = data.skip(int(0.3 * len(data)))


#### Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
def create_model():

In [ ]:
model = keras.Sequential([
    layers.Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(32, (3,3), 1, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, (3,3), 1, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
return model

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

#### Evaluate Model with Cross Validation

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
num_folds = 5
fold_acc = []
fold_precision = []
fold_recall = []

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=123)
for fold, (train_idx, val_idx) in enumerate(kfold.split(data)):
    train_data_fold = data.take(train_idx)
    val_data_fold = data.take(val_idx)
    model = create_model()
    history = model.fit(train_data_fold, epochs=10, validation_data=val_data_fold, verbose=0)
    _, acc = model.evaluate(val_data_fold, verbose=0)
    pre = Precision()
    re = Recall()
    for batch in val_data_fold.as_numpy_iterator(): 
        X, y = batch
        yhat = model.predict(X)
        pre.update_state(y, yhat)
        re.update_state(y, yhat)
    fold_acc.append(acc)
    fold_precision.append(pre.result().numpy())
    fold_recall.append(re.result().numpy())

#### Testing

In [ ]:
import cv2

In [ ]:
img = cv2.imread('Productivity.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Good')
else:
    print(f'Predicted class is Bad')

#### Saving the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
new_model = load_model('imageclassifier.h5')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))